### Importing modules

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint
import yfinance as yf
import requests
import bs4
from bs4 import BeautifulSoup
import time

### Gathering the Google Sheets API credentials locally and connecting them to the sheets to access data

In [ ]:
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
# Replace XXXX with the path to the google sheets api credentials (creds.json file that is saved locally)
path = r"XXXXX"
creds = ServiceAccountCredentials.from_json_keyfile_name(path + "creds.json",  scope, token_uri = "https://oauth2.googleapis.com/token")
client = gspread.authorize(creds)
sheet = client.open(r'XXXXX') # Replace XXXX with the google sheet name
sheet1 = sheet.worksheet("Company Financial analysis")
maxlen = 150  # Max number of records

def profiledata(ticker, page, attr, position):
    # This function takes 
    #       - ticker (eg: AAPL)
    #       - page (eg: financials or balance-sheet or key-statistics)
    #       - attr (eg: Net Income, Revenue etc from each of the pages. Works with most but not all attributes)
    #       - position (eg: 1, 2, 3 etc. What is the column number where the data is located)
    try:
        # Obtaining the source url based on the ticker, page etc.
        source = str('https://ca.finance.yahoo.com/quote/') + ticker + str('/') + page + str(r'?p=') + ticker
        # Accessing the url using the requests module
        r = requests.get(source)
        # Converting and accessing the url as an lxml using beautiful soup
        soup = bs4.BeautifulSoup(r.text,'lxml')
        # Try to find the div type in the lxml called rw-expanded if it exists
        rows = soup.find_all('div', {'class':'rw-expnded'})
        struct = 'div'
        # If data is tabular, it will be tr instead of div
        if len(rows)==0:
            rows = soup.find_all('tr')
            struct = 'tr'
        value = '-'
        # Looping through each div or tr section in the lxml and matching first text to the attr
        for i in rows:
            if i.find_all('span')[0].text == attr:
                if struct == 'div':
                    value = i.find_all('span')[position].text
                    break
                elif struct == 'tr':
                    value = i.find_all('td')[position].text
                    break
            else:
                value = "-"
    except:
        value = '-'
    return value


for i in range(123,maxlen):
    t = sheet1.cell(i+3,5).value  # Get the ticker from the 5th column
    if not len(t)==0:
        
        # Calling the profiledata function for all the following parameters
        rev2019 = profiledata(t, 'financials', 'Total Revenue', 2 )
        rev2018 = profiledata(t, 'financials', 'Total Revenue', 3 )
        rev2017 = profiledata(t, 'financials', 'Total Revenue', 4 )
        ebitda2019 = profiledata(t, 'financials', 'EBITDA', 1)
        ebitda2018 = profiledata(t, 'financials', 'EBITDA', 2)
        ebitda2017 = profiledata(t, 'financials', 'EBITDA', 3)
        netincome = profiledata(t, 'financials', 'Net Income', 2)
        interestexp = profiledata(t, 'financials', 'Interest Expense', 2)
        totliabilities = profiledata(t, 'balance-sheet', 'Total Liabilities', 1)
        totequities = profiledata(t, 'balance-sheet', "Total stockholders' equity", 1)
        p_e = profiledata(t, 'key-statistics', 'Trailing P/E', 1)

        # Updating various columns in the google sheet with the stock parameters obtained
        sheet1.update_cell(i+3,6,rev2019)
        sheet1.update_cell(i+3,7,rev2018)
        sheet1.update_cell(i+3,8,rev2017)
        sheet1.update_cell(i+3,9,ebitda2019)
        sheet1.update_cell(i+3,10,ebitda2018)
        sheet1.update_cell(i+3,11,ebitda2017)
        sheet1.update_cell(i+3,12,totliabilities)
        sheet1.update_cell(i+3,13,totequities)
        sheet1.update_cell(i+3,14,p_e)
        sheet1.update_cell(i+3,15,netincome)
        sheet1.update_cell(i+3,16,interestexp)
        
        # Sleeping after quering all the above to avoid crossing the google api read/write limits
        time.sleep(15)
    else:
        break